In [2]:
import MySQLdb
import sys
import bz2
import pandas as pd

In [3]:
################################# IrmaDB_results ###########################
# Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [4]:
################################# IrmaDB_prod(_backup) ####################
# Virtual machine IP adress
HOST2            = "localhost"

## MySQL user
DB_USER2         = "root"

## MySQL password
DB_PASSWD2       = "password"

In [6]:
# cids : list of 2 cids, the first one being a cid in the old database, and the second a cid in current the database
# Queries the size of the kernel of the two cids and return them in a list of two elements.
def get_sizes(cids):
    try:
        socket_new = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
        socket_old = MySQLdb.connect(HOST2, DB_USER2, DB_PASSWD2, "IrmaDB_prod_backup")
        cursor_new = socket_new.cursor()
        cursor_old = socket_old.cursor()
        # retrieve the size of first cid
        query_new="SELECT compiled_kernel_size FROM `compilations` where cid={}".format(cids[1])
        query_old="SELECT core_size FROM `Compilations` where cid={}".format(cids[0])
        cursor_new.execute(query_new)
        cursor_old.execute(query_old)
        result_new=cursor_new.fetchone()
        #print("select compiled_kernel_size from compilations where cid={} : {}".format(cids[1], result1))
        result_old=cursor_old.fetchone()
        #print("select core_size from Compilations where cid={} : {}".format(cids[0], result2))
        return [result_old[0], result_new[0]]
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database", file=sys.stderr)
        exit(-1)
    finally:
        cursor_new.close()
        cursor_old.close()
        socket_new.close()
        socket_old.close()

In [7]:
# This list contains the associations between cids in the old database ([0]) and the cid of their mutated .config 
# compilation in the new database.
# This list has been produced by parsing the output file of the jobs in charge of the new compilation which took 
# the ancient .config files from old database
l= [['100817', '74258'], ['101222', '74259'], ['101703', '74260'], ['101863', '74261'], \
    ['103074', '74262'], ['103430', '74263'], ['107624', '74264'], ['108678', '74265'], \
    ['109504', '74266'], ['109991', '74267'], ['110107', '74268'], ['110840', '74269'], \
    ['112869', '74270'], ['114194', '74271'], ['114560', '74272'], ['115083', '74273'], \
    ['116131', '74274'], ['30196', '74275'], ['30199', '74276'], ['30576', '74277'], \
    ['31167', '74278'], ['32131', '74279'], ['34406', '74280'], ['34827', '74281'], \
    ['36050', '74282'], ['38821', '74283'], ['39512', '74284'], ['39580', '74285'], \
    ['41985', '74286'], ['42391', '74287'], ['42915', '74288'], ['43228', '74289'], \
    ['43243', '74290'], ['44115', '74291'], ['44188', '74292'], ['47488', '74293'], \
    ['49677', '74294'], ['50322', '74295'], ['50460', '74296'], ['50900', '74297'], \
    ['52158', '74298'], ['52542', '74299'], ['53035', '74300'], ['53285', '74301'], \
    ['54022', '74302'], ['55305', '74303'], ['55333', '74304'], ['56481', '74305'], \
    ['57131', '74306'], ['57521', '74307'], ['59533', '74308'], ['59549', '74309'], \
    ['60417', '74310'], ['60969', '74311'], ['61410', '74312'], ['62122', '74313'], \
    ['62700', '74314'], ['65027', '74315'], ['66622', '74316'], ['66733', '74317'], \
    ['68416', '74318'], ['68438', '74319'], ['70567', '74320'], ['71741', '74321'], \
    ['72791', '74322'], ['73129', '74323'], ['73441', '74324'], ['73749', '74325'], \
    ['74557', '74326'], ['75541', '74327'], ['75796', '74328'], ['77330', '74329'], \
    ['78480', '74330'], ['79660', '74331'], ['79695', '74332'], ['81591', '74333'], \
    ['82072', '74334'], ['82330', '74335'], ['83256', '74336'], ['83842', '74337'], \
    ['84391', '74338'], ['84612', '74339'], ['84961', '74340'], ['85715', '74341'], \
    ['88408', '74342'], ['88747', '74343'], ['88988', '74344'], ['89691', '74345'], \
    ['89894', '74346'], ['91657', '74347'], ['92246', '74348'], ['92478', '74349'], \
    ['92557', '74350'], ['93850', '74351'], ['95587', '74352'], ['95624', '74353'], \
    ['97877', '74354'], ['98599', '74355'], ['99104', '74356'], ['99413', '74357']]
tmp=[]
for i in l:
    sizes=get_sizes(i)
    tmp.append({'cid_origin': i[0], 'size_origin': sizes[0], 'cid_mutated': i[1], 'size_mutated': sizes[1], \
                'difference': sizes[0]-sizes[1]})

res=pd.DataFrame(tmp)
print(res.columns)
print(res.index)
res=res[['cid_origin', 'cid_mutated', 'size_origin', 'size_mutated', 'difference']]
res.to_pickle('DF_size_diff')

def highlight_col(x):
    #colour dataframe according to 'difference' value
    mask =  res['difference'].between(1900000 , 2100000)
    mask2 = res['difference'] < 1900000
    mask3 = res['difference'] > 2100000    
    x = pd.DataFrame('', index=res.index, columns=res.columns)
    x.loc[mask, ['difference']] = 'background-color: lawngreen'
    x.loc[mask2,['difference']] = 'background-color: red'
    x.loc[mask3,['difference']] = 'background-color: green'
    return x
res.style.apply(highlight_col, axis=None)


Index(['cid_mutated', 'cid_origin', 'difference', 'size_mutated',
       'size_origin'],
      dtype='object')
RangeIndex(start=0, stop=100, step=1)


,cid_origin,cid_mutated,size_origin,size_mutated,difference
0,100817,74258,32144496,30096112,2048384
1,101222,74259,29852984,27775800,2077184
2,101703,74260,14344024,12326632,2017392
3,101863,74261,25778304,23777408,2000896
4,103074,74262,23348936,21349952,1998984
5,103430,74263,18742112,16702648,2039464
6,107624,74264,16932824,16938856,-6032
7,108678,74265,30506776,28565760,1941016
8,109504,74266,35563776,33656944,1906832
9,109991,74267,33761832,31799096,1962736
